In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# The dataset contains 2019 observations and 9 columns.
file_path = "..\data\ALGO_daily.csv"
df = pd.read_csv(file_path)
df.set_index(df.columns[0], inplace=True)
df.head()

,open,high,low,close,return,return2,RV,lnRV,lnRV_7D,lnRV_30D,lnRV_60D
timestamp,,,,,,,,,,,
2019-08-20,0.655,0.656,0.603,0.625,-0.046884,0.002198,0.007147,-4.941038,-5.120973,-4.842265,-4.657915
2019-08-21,0.626,0.627,0.559,0.567,-0.097392,0.009485,0.005644,-5.177089,-5.232313,-4.847512,-4.692676
2019-08-22,0.567,0.592,0.531,0.572,0.008780,0.000077,0.006631,-5.015983,-5.275639,-4.863660,-4.711858
2019-08-23,0.573,0.589,0.546,0.576,0.006969,0.000049,0.005480,-5.206578,-5.270696,-4.876759,-4.737760
2019-08-24,0.574,0.580,0.542,0.577,0.001735,0.000003,0.003904,-5.545689,-5.283678,-4.883515,-4.777490


In [ ]:
# create lag terms of lnRV
df['lnRV_lag1'] = df['lnRV'].shift(1)
df['lnRV_7D_lag1'] = df['lnRV_7D'].shift(1)
df['lnRV_30D_lag1'] = df['lnRV_30D'].shift(1)
df['lnRV_60D_lag1'] = df['lnRV_60D'].shift(1)
df.dropna()
df.head()

,open,high,low,close,return,return2,RV,lnRV,lnRV_7D,lnRV_30D,lnRV_60D,lnRV_lag1,lnRV_7D_lag1,lnRV_30D_lag1,lnRV_60D_lag1
timestamp,,,,,,,,,,,,,,,
2019-08-20,0.655,0.656,0.603,0.625,-0.046884,0.002198,0.007147,-4.941038,-5.120973,-4.842265,-4.657915,NaN,NaN,NaN,NaN
2019-08-21,0.626,0.627,0.559,0.567,-0.097392,0.009485,0.005644,-5.177089,-5.232313,-4.847512,-4.692676,-4.941038,-5.120973,-4.842265,-4.657915
2019-08-22,0.567,0.592,0.531,0.572,0.008780,0.000077,0.006631,-5.015983,-5.275639,-4.863660,-4.711858,-5.177089,-5.232313,-4.847512,-4.692676
2019-08-23,0.573,0.589,0.546,0.576,0.006969,0.000049,0.005480,-5.206578,-5.270696,-4.876759,-4.737760,-5.015983,-5.275639,-4.863660,-4.711858
2019-08-24,0.574,0.580,0.542,0.577,0.001735,0.000003,0.003904,-5.545689,-5.283678,-4.883515,-4.777490,-5.206578,-5.270696,-4.876759,-4.737760
